In [3]:
import os
from pathlib import Path
import pandas as pd
# import numpy as np
from sqlalchemy import create_engine,text
# from scipy import stats
import plotly.express as px
import toml
import sys

sys.path.append("../../notebook_styling")
import psrc_theme

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# emme_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','emme_configuration.toml'))
# network_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','network_configuration.toml'))

# table format
pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.1%}".format

In [4]:
# summary data location
df_result = pd.read_csv(Path(valid_config['model_dir']) / 'outputs/network/network_results.csv')

# observed counts location
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])
df_observed_bike = pd.read_sql(text("SELECT * FROM observed_bike_2023"), con=conn.connect())

In [5]:
df_result[df_result['@countid'] == 2046][['tod','@bvol']].to_clipboard()

In [6]:
df = df_result[['@bvol','@countid']].groupby('@countid').sum()[['@bvol']]

In [7]:
df = df.merge(df_observed_bike, left_on='@countid', right_on='countID', suffixes=('', '_obs'))

In [8]:
df = df.groupby('countID').sum()[['location','@bvol','observed_counts']]

In [17]:
df[['@bvol','observed_counts']] = df[['@bvol','observed_counts']].apply(pd.to_numeric, errors='coerce').astype('int')

In [18]:
# Create scatter plot of @bvol vs observed_counts
fig = px.scatter(df, x='@bvol', y='observed_counts',
                 labels={'location': 'location', '@bvol': 'Simulated Bike Volume', 'observed_counts': 'Observed Bike Volume'},
                 title='Simulated vs Observed Bike Volumes',
                 # Hover data show location
                 hover_data={'location': True, '@bvol': True, 'observed_counts': True}, 
                 # set axis range to be equal on both axes
                 range_x=[0, df[['@bvol', 'observed_counts']].max().max()*1.05],
                 range_y=[0, df[['@bvol', 'observed_counts']].max().max()*1.05],

)
fig.show()